In [1]:
from pyzotero import zotero
import os
import sys
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm.notebook import tqdm
tqdm.pandas()

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from code_utils.zotero import get_data_from_zotero
from code_utils.glutton import get_doi_glutton
from code_utils.utils import get_doi_cleaned,aplatir,wg_chap_to_dict,get_year_ipbes,check_doi_glutton
from code_utils.pickle import load_cache,write_cache
from code_utils.enriching_data_OpenAlex import get_countries_concepts_sdg,get_open_alex_data,get_open_alex_data_not_in_references

In [2]:
cached_openalex_data_ipbes = {}
cached_openalex_data_not_ipbes = {}

In [3]:
try:
    cached_openalex_data_ipbes = load_cache(cached_openalex_data_ipbes,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data_ipbes.pkl')
    cached_openalex_data_not_ipbes = load_cache(cached_openalex_data_ipbes,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data_not_ipbes.pkl')
except:
    #write_cache(cached_openalex_data_ipbes,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data_ipbes.pkl')
    #write_cache(cached_openalex_data_not_ipbes,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data_not_ipbes.pkl')
    print('oups')

6607 data in cached openalex data
70 data in cached openalex data


Read IPBES references in zotero files

In [ ]:
ZOTERO_KEY = os.getenv('ZOTERO_KEY')
IPBES_ZOTERO_ID = '2333077'
COLLECTION_IDS = ['8DQ8YFJI','JR5LKU4U','LBFNF62B','7IMYH9U3','DRZLUY9K','KTPTCAVF','UWNSUNXQ','BJGWRJAK','JT8FJXP6']
CHAPTER_IDS=['1','2.1','2.2','2.3','3','4','5','6','glossary']
ALL_COLLECTIONS={}

In [ ]:
ALL_COLLECTIONS=get_data_from_zotero(ZOTERO_KEY,IPBES_ZOTERO_ID,COLLECTION_IDS,CHAPTER_IDS,ALL_COLLECTIONS)

In [ ]:
df_ipbes_chap={}
for i in range(len(COLLECTION_IDS)):
    df_ipbes_chap[CHAPTER_IDS[i]]=pd.DataFrame(list(pd.DataFrame(ALL_COLLECTIONS[f'collection_chapter_{CHAPTER_IDS[i]}']).data))[['DOI','title','date','creators','tags']]
    df_ipbes_chap[CHAPTER_IDS[i]]['chap']=f'chap_{CHAPTER_IDS[i]}'
    print(f'{len(df_ipbes_chap[CHAPTER_IDS[i]][~pd.isna(df_ipbes_chap[CHAPTER_IDS[i]].DOI)])/len(df_ipbes_chap[CHAPTER_IDS[i]])*100} % doi available for chapter {CHAPTER_IDS[i]}')

Clean and enrich data 

In [ ]:
df_ipbes=pd.concat(list(df_ipbes_chap.values())).reset_index()
del df_ipbes['index']

In [ ]:
len(df_ipbes[~pd.isna(df_ipbes.DOI)])/len(df_ipbes)*100

In [ ]:
get_doi_glutton(df_ipbes.iloc[1,:])

In [ ]:
df_ipbes.loc[pd.isna(df_ipbes.DOI),'DOI']=df_ipbes.loc[pd.isna(df_ipbes.DOI),:].progress_apply(get_doi_glutton, axis=1)

In [ ]:
len(df_ipbes[~pd.isna(df_ipbes.DOI)])/len(df_ipbes)*100

In [ ]:
df_ipbes.loc[:,'DOI']=df_ipbes.loc[:,'DOI'].apply(lambda x: get_doi_cleaned(x))
df_ipbes.loc[:,'DOI']=df_ipbes.loc[:,'DOI'].apply(lambda x: None if str(x)[:4]=='http' else x)
df_ipbes['freq']=1
df_ipbes=df_ipbes.rename(columns={'DOI':'doi'})

Enriching data through OpenAlex API integration

In [ ]:
df_unique_doi=df_ipbes.groupby(by=['doi'], dropna=True).agg({'chap': lambda x: list(x), 'freq': 'sum'})
df_unique_doi=df_unique_doi.reset_index()

In [ ]:
#get data_OpenAlex from the API 
df_unique_doi.apply(lambda row: get_open_alex_data(cached_openalex_data_ipbes,row['doi']), axis= 1)

In [ ]:
len(cached_openalex_data_ipbes)

In [ ]:
#write_cache(cached_openalex_data_ipbes,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data_ipbes.pkl')

In [41]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes_visualization.json')

In [42]:
df_unique_doi=df_ipbes['doi'].drop_duplicates().dropna()
df_unique_doi=pd.DataFrame({'doi':list(df_unique_doi)})

In [43]:
countries_list = []
concepts_list = []
sdg_list = []
topics_list = []
is_OA_available_list=[]
title_list=[]
year_list=[]
names_list=[]
rors_list=[]
institutions_names_list=[]
locations_list=[]

for i,row in df_unique_doi.iterrows():
    countries,concepts,sdg,year,topics,doi,bool_OA,title,name,rors,institutions_names,locations_names=get_countries_concepts_sdg(cached_openalex_data_ipbes,row,ipcc=True,i=0)

    countries_list.append(countries)
    concepts_list.append(concepts)
    sdg_list.append(sdg)
    topics_list.append(topics)  
    is_OA_available_list.append(bool_OA)
    title_list.append(title)
    year_list.append(year)
    names_list.append(name)
    rors_list.append(rors)
    institutions_names_list.append(institutions_names)
    locations_list.append(locations_names)

df_unique_doi['countries'] = countries_list
df_unique_doi['concepts'] = concepts_list
df_unique_doi['sdg'] = sdg_list
df_unique_doi['topics'] = topics_list
df_unique_doi['is_OA_available'] = is_OA_available_list
df_unique_doi['year_OA'] = year_list
df_unique_doi['authors_name'] = names_list
df_unique_doi['rors'] = rors_list
df_unique_doi['institutions_names'] = institutions_names_list
df_unique_doi['locations_names'] = locations_list

In [44]:
df_unique_doi.is_OA_available.value_counts()

is_OA_available
True     6581
False      24
Name: count, dtype: int64

In [45]:
df_unique_doi=df_unique_doi[df_unique_doi.is_OA_available]

In [46]:
df_unique_doi=df_unique_doi.reset_index()
del df_unique_doi['index']

In [12]:
df_unique_doi.to_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes_visualization2.json')

Visualization of the contribution of each countries in ipcc references

In [47]:
df_ipbes=pd.merge(df_ipbes[['doi', 'title', 'date', 'creators', 'tags']], df_unique_doi, on='doi', how='inner')

In [48]:
df_ipbes

,doi,title,date,creators,tags,countries,concepts,sdg,topics,is_OA_available,year_OA,authors_name,rors,institutions_names,locations_names
0,10.1890/1051-0761(2000)010[1251:roteka]2.0.co;2,REDISCOVERY OF TRADITIONAL ECOLOGICAL KNOWLEDG...,10/2000,"[{'creatorType': 'author', 'firstName': 'Fikre...","[{'tag': 'ipbes-global_chapter1'}, {'tag': 'ip...","[CA, SE]","[{'name': 'Traditional knowledge'}, {'name': '...","[{'id': '15', 'name': 'Life on land'}]",[{'name': 'Drivers and Impacts of Tropical Def...,True,2000.0,"[{'Fikret Berkes': ['CA']}, {'Johan Colding': ...","[[{'https://ror.org/02gfys938': 'CA'}], [{'htt...","[[{'University of Manitoba': 'CA'}], [{'Royal ...",[Ecological Applications]
1,10.1890/1051-0761(2000)010[1251:roteka]2.0.co;2,REDISCOVERY OF TRADITIONAL ECOLOGICAL KNOWLEDG...,10/2000,"[{'creatorType': 'author', 'firstName': 'Fikre...","[{'tag': 'ipbes-global_chapter1'}, {'tag': 'ip...","[CA, SE]","[{'name': 'Traditional knowledge'}, {'name': '...","[{'id': '15', 'name': 'Life on land'}]",[{'name': 'Drivers and Impacts of Tropical Def...,True,2000.0,"[{'Fikret Berkes': ['CA']}, {'Johan Colding': ...","[[{'https://ror.org/02gfys938': 'CA'}], [{'htt...","[[{'University of Manitoba': 'CA'}], [{'Royal ...",[Ecological Applications]
2,10.1890/1051-0761(2000)010[1251:roteka]2.0.co;2,REDISCOVERY OF TRADITIONAL ECOLOGICAL KNOWLEDG...,10/2000,"[{'creatorType': 'author', 'firstName': 'Fikre...","[{'tag': 'ipbes-global_chapter1'}, {'tag': 'ip...","[CA, SE]","[{'name': 'Traditional knowledge'}, {'name': '...","[{'id': '15', 'name': 'Life on land'}]",[{'name': 'Drivers and Impacts of Tropical Def...,True,2000.0,"[{'Fikret Berkes': ['CA']}, {'Johan Colding': ...","[[{'https://ror.org/02gfys938': 'CA'}], [{'htt...","[[{'University of Manitoba': 'CA'}], [{'Royal ...",[Ecological Applications]
3,10.1016/j.gloenvcha.2013.12.012,Evaluating knowledge exchange in interdiscipli...,2014,"[{'creatorType': 'author', 'firstName': 'Ioan'...",[{'tag': 'ipbes-global_chapter1'}],[GB],"[{'name': 'Knowledge management'}, {'name': 'S...","[{'id': '15', 'name': 'Life on land'}]",[{'name': 'Sustainability Transitions and Resi...,True,2014.0,"[{'Ioan Fazey': ['GB']}, {'Lukas Bunse': ['GB'...","[[{'https://ror.org/03h2bxq36': 'GB'}], [{'htt...","[[{'University of Dundee': 'GB'}], [{'Universi...",[Global Environmental Change]
4,10.5751/es-07868-200344,Multiple telecouplings and their complex inter...,2015,"[{'creatorType': 'author', 'firstName': 'Jiang...","[{'tag': 'ipbes-global_chapter1'}, {'tag': 'ip...",[],"[{'name': 'Zhàng'}, {'name': 'Ecology'}, {'nam...",None,[{'name': 'Global Analysis of Ecosystem Servic...,True,2015.0,"[{'Jianguo Liu': []}, {'Vanessa Hull': []}, {'...","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...","[Ecology and Society, DOAJ (DOAJ: Directory of..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7501,10.1016/b978-0-444-52512-3.00231-x,Domestication of Plants,2014,"[{'creatorType': 'author', 'firstName': 'P.', ...",[{'tag': 'ipbes-global_glossary'}],[US],"[{'name': 'Domestication'}, {'name': 'Agricult...","[{'id': '2', 'name': 'Zero hunger'}]",[{'name': 'Impact of Pollinator Decline on Eco...,True,2014.0,[{'Paul Gepts': ['US']}],[[{'https://ror.org/05rrcem69': 'US'}]],"[[{'University of California, Davis': 'US'}]]",[Elsevier eBooks]
7502,10.1016/s0021-8502(99)00571-6,"Aerosol technology : properties, behavior, and...",1999,"[{'creatorType': 'author', 'firstName': 'Willi...",[{'tag': 'ipbes-global_glossary'}],[US],"[{'name': 'Aerosol'}, {'name': 'Environmental ...","[{'id': '7', 'name': 'Affordable and clean ene...",[{'name': 'Estimating Vehicle Fuel Consumption...,True,2000.0,[{'M.L. Laucks': ['US']}],[[{'https://ror.org/00cvxb145': 'US'}]],[[{'University of Washington': 'US'}]],[Journal of Aerosol Science]
7503,10.1201/9781315153292-16,Integrated Water Resources Management in Action,2009,"[{'creatorType': 'author', 'firstName': 'Jan',...",[{'tag': 'ipbes-gl

In [49]:
df_ipbes['year']=df_ipbes['date'].apply(lambda x: get_year_ipbes(x) if pd.isna(x)==False else None)

In [52]:
df_ipbes['check']=df_ipbes.apply(lambda row: check_doi_glutton(row), axis=1)
df_ipbes.check.value_counts()

check
True     6938
False     568
Name: count, dtype: int64

In [53]:
df_ipbes=df_ipbes[(df_ipbes.check) & (pd.isna(df_ipbes['is_OA_available'])==False)]

In [54]:
del df_ipbes['check']

In [57]:
df_ipbes.loc[:,'countries']=df_ipbes.loc[:,'countries'].apply(lambda x: str(x).replace('nan','None').replace('[]','None').replace('[None]','None') if ((str(x)=='nan')|(str(x)=='[None]')|((str(x)=='[]'))) else x )
df_ipbes.loc[:,'countries']=df_ipbes.loc[:,'countries'].apply(lambda x: ['None'] if x=='None' else x)

In [59]:
df_ipbes.to_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes_visualization.json', orient='records')

In [ ]:
df_ipbes.columns

In [ ]:
data_counts = pd.Series(aplatir(list(df_ipbes['countries']))).value_counts().drop('None')

In [ ]:
plt.figure(figsize=(10, 6))
data_counts[:20].plot(kind='bar')
ax = data_counts[:20].plot(kind='bar')
for i, v in enumerate(data_counts[:20]):
    ax.text(i, v + 0.1, str(v), ha='center', va='bottom')
plt.show()

Find the data for constructing the learning model ( IPBES related or not )

In [ ]:
df_ipbes['year'] = pd.to_numeric(df_ipbes['year'], errors='coerce')
year=df_ipbes.drop_duplicates(subset='doi')['year'].dropna().sort_values().apply(lambda x: int(x))
year_counts = pd.Series(list(year)).value_counts().to_dict()

In [ ]:
dois=list(df_unique_doi.doi)

In [ ]:
year_counts_not_ipbes={}
target_iterations = len(dois)

In [ ]:
for year in list(year_counts.keys()):   
    cached_openalex_data_not_ipbes[year]=[]
    year_counts_not_ipbes[year]=0
    while year_counts_not_ipbes[year]<year_counts[year]:
        get_open_alex_data_not_in_references(dois,cached_openalex_data_not_ipbes,year_counts,year_counts_not_ipbes,year)
    cached_openalex_data_not_ipbes[year]=cached_openalex_data_not_ipbes[year][:year_counts[year]+1]

In [ ]:
len(cached_openalex_data_not_ipbes)

In [ ]:
write_cache(cached_openalex_data_not_ipbes,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data_not_ipbes.pkl')

In [ ]:
df_not_ipbes=pd.DataFrame()
year_list = []
countries_list = []
concepts_list = []
sdg_list = []
topics_list = []
dois_list = []
for k in range(len(aplatir(list(cached_openalex_data_not_ipbes.values())))):
    countries,concepts,sdg,year,topics,doi=get_countries_concepts_sdg(cached_openalex_data=aplatir(list(cached_openalex_data_not_ipbes.values())),ipcc=False,i=k)

    countries_list.append(countries)
    concepts_list.append(concepts)
    sdg_list.append(sdg)
    year_list.append(year)   
    topics_list.append(topics)   
    dois_list.append(doi)   

df_not_ipbes['countries'] = countries_list
df_not_ipbes['concepts'] = concepts_list
df_not_ipbes['sdg'] = sdg_list
df_not_ipbes['year'] = year_list
df_not_ipbes['topics'] = topics_list
df_not_ipbes['doi'] = dois_list

In [ ]:
df_not_ipbes

In [ ]:
#df_not_ipbes.to_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_not_ipbes.jsonl', orient='records', lines= True)